# Store Item Demand Forecasting - Final Results & Analysis

このノートブックは、これまでの実験の集大成であり、今後のモデルチューニングの「基準点（ベースライン）」となるものです。

### 達成したこと
1. **数学的エンコード**: 曜日の円環構造（Sin/Cos）を導入し、モデルに幾何学的な周期性を教えました。
2. **ロジックの分離**: `src/` フォルダへアルゴリズムを抽出し、クリーンな開発環境を構築しました。
3. **精度向上**: SMAPEスコアで良好なベースライン（12.43）を記録しました。

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

# プロジェクトルートを作業ディレクトリに追加
sys.path.append("/work/competitions/demand-forecasting-kernels-only/")

from src.features import create_date_features, create_lag_features, create_rolling_features
from src.models import train_lgbm, smape

%matplotlib inline
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (15, 6)

## 1. データの準備とモデルの訓練

数学的構造（Cyclic Encoding）を採用した最終構成でモデルを動かします。

In [ ]:
DATA_DIR = "/work/competitions/demand-forecasting-kernels-only/data"
train_raw = pd.read_csv(f"{DATA_DIR}/train.csv", parse_dates=["date"])

# 特徴量生成 (srcから呼び出し)
print("Creating features...")
df = create_date_features(train_raw)
df = create_lag_features(df, lags=[91, 98, 105, 112])
df = create_rolling_features(df, windows=[30, 90], shift=91)
df = df.dropna()

# 時系列分割
split_date = "2017-10-01"
train_df = df[df["date"] < split_date]
val_df = df[df["date"] >= split_date]

features = [
    "store", "item", "dayofweek_sin", "dayofweek_cos", "month_sin", "month_cos",
    "sales_lag_91", "sales_lag_98", "sales_lag_105", "sales_lag_112",
    "rolling_mean_30", "rolling_mean_90"
]

print(f"Training with {len(features)} features...")
model = train_lgbm(train_df[features], train_df["sales"], val_df[features], val_df["sales"])

# 予測の取得
val_df = val_df.copy()
val_df["preds"] = model.predict(val_df[features])
print(f"\nFinal Validation SMAPE: {smape(val_df['preds'], val_df['sales']):.6f}")

## 2. 視覚的な検証

### 2.1 実績値 vs 予測値 (Time Series Plot)

モデルが実際の売上のリズム（特に週次・年次の波）をどれくらい正確に捉えられているかを、Store 1 / Item 1 を例に確認します。

In [ ]:
sample = val_df[(val_df["store"] == 1) & (val_df["item"] == 1)]

plt.figure(figsize=(15, 6))
plt.plot(sample["date"], sample["sales"], label="Actual Sales", color="royalblue", marker="o", markersize=4, alpha=0.7)
plt.plot(sample["date"], sample["preds"], label="Predicted Sales", color="crimson", linestyle="--", linewidth=2)
plt.title("Actual vs Predicted Sales (Store 1, Item 1) - Last 3 Months")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.show()

### 2.2 誤差の分析 (Error Distribution)

予測誤差が正規分布に近いか、あるいは偏りがあるかを確認します。

In [ ]:
val_df["error"] = val_df["preds"] - val_df["sales"]

plt.figure(figsize=(10, 5))
sns.histplot(val_df["error"], bins=50, kde=True, color="teal")
plt.axvline(x=0, color="red", linestyle="--")
plt.title("Error Distribution (Residuals)")
plt.xlabel("Prediction Error (Preds - Actual)")
plt.show()

## 3. 特徴量重要度の深掘り

Gain（情報利得）と Split（分割数）の両方を確認することで、どの特徴量が「決定的な判断」に使われたかが見えてきます。

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

lgb.plot_importance(model, importance_type="gain", ax=ax1, title="Feature Importance (Gain)")
lgb.plot_importance(model, importance_type="split", ax=ax2, title="Feature Importance (Split)")

plt.tight_layout()
plt.show()

## 4. 今後のチューニングに向けたヒント

このモデルをさらに改善するための「数学修士のアイデアリスト」です。

| 手法 | 期待される効果 | 難易度 |
| :--- | :--- | :--- |
| **Optunaによるチューニング** | `num_leaves` や `learning_rate` の最適化。 | 低 (自動化可能) |
| **祝日フラグの追加** | 売上が特異的に変化する日の学習。 | 中 (外部データが必要) |
| **月・日の円環構造の精緻化** | `dayofyear_sin/cos` の追加など。 | 低 (幾何学的アプローチ) |
| **アンサンブル** | XGBoostやCatBoostとの加重平均。 | 中 (計算コスト増) |